In [1]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import numpy as np
import pandas as pd
import nltk
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag
from sklearn.model_selection import train_test_split
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import joblib
from collections import Counter
from tensorflow.keras.models import load_model

In [2]:
def load_file_to_string(file_path):
    with open(file_path, 'r') as file:
        text = file.read()
    return text

file_path = 'predict.txt'
file_contents = load_file_to_string(file_path)


In [3]:
file_contents

'computers consist of electronical components.\niam using windows 10 because i dont like windows 11.'

In [4]:
stop = list(stopwords.words('english'))
def tokenize(text):
    text = text.lower()
    text = text.replace("\\n", "")
    text = text.replace("\\", "")
    text = text.replace("'", "")
    pattern = r"^[a-z]+(-[a-z]+)*.?$"
    tokens = word_tokenize(text)
    tokens = [token for token in tokens if  re.match(pattern, token)]
    tokens = [x for x in tokens if x not in stop]
    
    return list(tokens)

text_token = tokenize(file_contents)

In [5]:
tag_map = {
    "N": "n",
    "V": "v",
    "R": "r",
    "J": "a"
}

def pos_lemma(tokens):
    tags=pos_tag(tokens)
    lemmatizer=WordNetLemmatizer()
    w_net_lemma=[]
    for token ,tag in tags:
        if(tag_map.get(tag[0]) is not None ):
            w_net_lemma.append(lemmatizer.lemmatize(token,pos=tag_map[tag[0]]))
        else:
            w_net_lemma.append(lemmatizer.lemmatize(token,pos='n'))
            
    return list(w_net_lemma)

text_lemma = pos_lemma(text_token)

In [6]:
tfidf_vect = joblib.load("pk1/general_tfidf.joblib")
text_tfidf =  tfidf_vect.transform(text_lemma)


In [7]:
class_dict = {
    0: 'alt.atheism',
    1: 'comp.graphics',
    2: 'comp.os.ms-windows.misc',
    3: 'comp.sys.ibm.pc.hardware',
    4: 'comp.sys.mac.hardware',
    5: 'comp.windows.x',
    6: 'misc.forsale',
    7: 'rec.autos',
    8: 'rec.motorcycles',
    9: 'rec.sport.baseball',
    10: 'rec.sport.hockey',
    11: 'sci.crypt',
    12: 'sci.electronics',
    13: 'sci.med',
    14: 'sci.space',
    15: 'soc.religion.christian',
    16: 'talk.politics.guns',
    17: 'talk.politics.mideast',
    18: 'talk.politics.misc',
    19: 'talk.religion.misc'
}

<!-- Class is alt.atheism and its number is 0
Class is comp.graphics and its number is 1
Class is comp.os.ms-windows.misc and its number is 2
Class is comp.sys.ibm.pc.hardware and its number is 3
Class is comp.sys.mac.hardware and its number is 4
Class is comp.windows.x and its number is 5
Class is misc.forsale and its number is 6
Class is rec.autos and its number is 7
Class is rec.motorcycles and its number is 8
Class is rec.sport.baseball and its number is 9
Class is rec.sport.hockey and its number is 10
Class is sci.crypt and its number is 11
Class is sci.electronics and its number is 12
Class is sci.med and its number is 13
Class is sci.space and its number is 14
Class is soc.religion.christian and its number is 15
Class is talk.politics.guns and its number is 16
Class is talk.politics.mideast and its number is 17
Class is talk.politics.misc and its number is 18
Class is talk.religion.misc and its number is 19 -->

In [8]:
NN_model = load_model("pk1/NN_model.h5")
text_tfidf_reshaped = text_tfidf.toarray()
pred = NN_model.predict(text_tfidf_reshaped)
pred = pred[0]
pred = np.argmax(pred)
print(f"Predictions for NN is {class_dict[pred]}")


1/1 [==============================] - 1s 544ms/step
Predictions for NN is comp.sys.ibm.pc.hardware
